In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
from time import localtime, strftime

import lutorpy as lua
require("nn")
require("optim")
require("cutorch")
require("cunn")

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 7)

# Number of samples in each class

TEST_SAMPLE_NUMBER = 51731
TRAIN_SAMPLE_NUMBER = 257187

In [2]:
ZERO_PADDING = 6

def get_filename(index):
    return "src/Csv/{}.txt".format(str(index).zfill(ZERO_PADDING))

def get_framename(index):
    return "src/Frame/{}.jpg".format(str(index).zfill(ZERO_PADDING))

NUMBER_OF_PEDESTRIANS = 12273

def download_pedestrian(index):
    assert(0 <= index < NUMBER_OF_PEDESTRIANS), "pedestrian number should be between 0 and {max}; given number: {id}".format(
        max=NUMBER_OF_PEDESTRIANS-1, id=index)
    filename = get_filename(index)
    data = pd.read_csv(filename, index_col=0)
    return data

In [3]:
def make_view(data, length, only_odd_rows=True):
    data = np.array(data).flatten()
    len1 = len(data)
    len2 = length
    return np.lib.stride_tricks.as_strided(data, shape=(len1 - len2 + 1, len2),
                                                    strides=(data.dtype.itemsize,) * 2)[::2 if only_odd_rows else 1, :]

In [ ]:
# array1: [sX1; sY1; sX2; sY2; ...]
# array2: [eX1; eY1; eX2; eY2; ...]
# output: [dist((sX1, sY1), (eX1, eY1)), dist((sX1, sY1), (eX1, eY1)),
#          dist((sX2, sY2), (eX2, eY2)), dist((sX2, sY2), (eX2, eY2)), ...]

def distance_for_each_point(array1, array2):
    array_length = len(array1)
    len2 = len(array2)
    assert (array_length == len2), "Arrays' sizes have to be equal (array1: {}, array2: {})".format(array_length, len2)
    
    if array1.ndim == 1:
        distance = np.linalg.norm((array1 - array2).reshape((int(array_length / 2), 2)), axis=1)
        result = np.array([[d, d] for d in distance]).flatten()
    else:
        result = np.array([distance_for_each_point(array1[i], array2[i]) for i in range(array_length)])
    
    return result

# metrics between real results (in tests) and predicted
def distance(test_results, predicted_results):
    return distance_for_each_point(np.array(test_results), predicted_results).sum() / TEST_SAMPLE_NUMBER / 2

# score between real results (in tests) and predicted
def score(test_results, predicted_results, baseline_results):
    test_results = np.array(test_results)
    predicted_results = np.array(predicted_results)
    baseline_results = np.array(baseline_results)
    return (1 - ((test_results - predicted_results) ** 2).sum()/((test_results - baseline_results) ** 2).sum())

In [ ]:
# data may be present as [n_features * n_samples] or [n_samples * n_features] 
# usually algorithms require second variant but I prefer first

def to_model(df):
    df = np.array(df).T
    return df.reshape((int(df.shape[0] / 2), int(df.shape[1] * 2)))

def to_model_with_features(df):
    df = np.array(df)
    movement_vector = np.diff(df,axis=0)
    speed = np.power(movement_vector, 2)
    speed[:,::2] = -speed[:,::2]
    speed = np.sqrt(np.diff(speed)[:,::2])
    double_speed = np.zeros_like(movement_vector, dtype=np.float64)
    double_speed[:,::2] = double_speed[:,1::2] = speed
    last_speed = speed[-1:]
    mean_speed = np.mean(speed, axis=0)
    cort_coord = movement_vector / double_speed
    angle = np.arctan2(cort_coord[:,::2], cort_coord[:,1::2]) / np.pi * 4
    
    return (np.concatenate((to_model(df), to_model(movement_vector), speed.T, last_speed.T, np.array([mean_speed]).T,
                            angle.T), axis=1)).astype(int)
def from_model(npa):
    return npa.reshape((int(npa.shape[0] * 2), int(npa.shape[1] / 2))).T

In [6]:
def make_delta(data, results):
    data = np.array(data)
    results = np.array(results)
    
    delta_results = np.copy(results)
    delta_results[0,:] -= data[-1,:]
    delta_results[1:,:] -= results[:-1,:]
    
    return delta_results

def unmake_delta(data, delta):
    result = np.copy(np.array(delta))
    
    for i, row in enumerate(result):
        if i == 0:
            result[i] += np.array(data)[-1,:]
        else:
            result[i] += result[i-1]
            
    return result

In [7]:
# first simply algorithm to get some start
# more about it you can find in 'baseline_distance_between_real_points_and_predicted.ipynb'
def baseline(test_data, start_point_index=0, number_of_points_to_return=5):
    error_template = "Start point index should be less than last point. Start point index: {st}, last point index: {end}"
    assert (start_point_index < len(test_data) - 1), error_template.format(st=start_point_index, end=len(test_data) - 1)
    
    start_point = np.array(test_data)[start_point_index]
    last_but_one_point = np.array(test_data)[-2]
    end_point = np.array(test_data)[-1]
    
    distance = distance_for_each_point(end_point, start_point)
    normalized_motion_vector = (end_point - start_point) / distance
    normalized_motion_vector[np.where(distance == 0)] = 0
    last_vector_length = distance_for_each_point(end_point, last_but_one_point)
    
    motion_vector = normalized_motion_vector * last_vector_length
    result = []
    for i in range(number_of_points_to_return):
        result.append(end_point + (i + 1) * motion_vector)
        
    return np.array(result)

In [8]:
def compare_results(function, test_results, parameter_name, list_of_values, **other_parameters):
    result = [] # we will keep results for each configuration here

    for i, value in enumerate(list_of_values):
        print("{cur}/{num}: {t}".format(cur=i+1, num=len(list_of_values), t=strftime("%Y-%m-%d %H:%M:%S", localtime())))
        other_parameters[parameter_name] = value        
        predicted_results = function(**other_parameters)
        np.save("src/Logs/{date}_predicted_coordinates_{function_name}_{parameter_name}_".format(
                date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name)
                + str(value), predicted_results)
        result.append(score(test_results, predicted_results, baseline(other_parameters['test_data'], start_point_index=2)))
        
    print("done! {time}".format(time=strftime("%Y-%m-%d %H:%M:%S", localtime())))
    print("Results: {}".format(result))
        
    ind = list_of_values
    number = len(ind)
    width = 2 / number
    result_bar = plt.bar(range(number), result, width, color='g')

    plt.ylabel('Average difference')
    plt.xlabel(parameter_name)
    plt.title("Difference between real points and predicted by {parameter_name} in {function_name}".format(
        function_name=function.__name__, parameter_name=parameter_name))
    plt.xticks(np.array(range(number)) + width/2, ind)
    plt.savefig("src/Plots/{date}_{function_name}_score_by_{parameter_name}_with_features.png".format(
        date=strftime("%Y%m%d", localtime()), function_name=function.__name__, parameter_name=parameter_name))    

In [9]:
# reading data from .csv files

test_data = pd.read_csv('src/test_data_points.csv', index_col=0)
train_data = pd.read_csv('src/train_data_points.csv', index_col=0)

# split ten frames in input and output data (we want to predict output by input)

test_results = test_data[5:10]
test_data = test_data[:5]

train_results = train_data[5:10]
train_data = train_data[:5]

In [53]:
# for transfer function you could use some of these: https://github.com/torch/nn/blob/master/doc/transfer.md

def mult_layers_nn(train_data, train_results, test_data, hidden_nodes_num, learning_rate, epoch_num):
    train_data_model = to_model_with_features(train_data)
    train_results_model = to_model(make_delta(train_data, train_results))
    test_data_model = to_model_with_features(test_data)
    
    ninputs = to_model_with_features(test_data).shape[1]
    noutputs = 10
    
    # define model (predictor): 
    transfer_function=nn.Tanh()    
    mlp = nn.Sequential()
    mlp._add(nn.Linear(ninputs, hidden_nodes_num)) 
    mlp._add(transfer_function)
    mlp._add(nn.Linear(hidden_nodes_num, hidden_nodes_num))
    mlp._add(transfer_function)
    mlp._add(nn.Linear(hidden_nodes_num, hidden_nodes_num))
    mlp._add(transfer_function)
    mlp._add(nn.Linear(hidden_nodes_num, hidden_nodes_num))
    mlp._add(transfer_function)
    mlp._add(nn.Linear(hidden_nodes_num, noutputs))
    mlp._cuda()

    # define a loss function to be minimized (mean-square error between predictions and groundtruth labels)
    criterion = nn.MSECriterion() 
    criterion._cuda()
    
    crit_change = []
    
    for epoch in range(epoch_num):
        # get sample
        inputs = (torch.fromNumpyArray(train_data_model.astype(float)))._cuda()
        outputs = (torch.fromNumpyArray(train_results_model.astype(float)))._cuda()

        # feed it to the neural network and the criterion
        crit_change.append(criterion._forward(mlp._forward(inputs), outputs))

        # train over this example in 3 steps
        # (1) zero the accumulation of the gradients
        mlp._zeroGradParameters()
        # (2) accumulate gradients
        mlp._backward(inputs, criterion.backward(criterion, mlp.output, outputs))

        # (3) update parameters with a learning rate
        mlp._updateParameters(learning_rate)
        
    result = (mlp._forward((torch.fromNumpyArray(test_data_model.astype(float)))._cuda())).asNumpyArray()
    
    crit_change = np.array(crit_change)
    print(crit_change[::int(epoch_num / 10)])
        
    return unmake_delta(test_data, from_model(result))

In [54]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=0.0001, epoch_num=100)

[ 549.76611328  547.04003906  546.90020752  546.76025391  546.62310791
  546.41265869  546.2020874   546.00024414  545.81085205  545.58612061]


In [55]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=0.001, epoch_num=100)

[ 549.66156006  544.44030762  543.11364746  542.11651611  542.77307129
  542.91906738  542.71173096  543.13085938  543.33959961  538.88330078]


In [56]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=0.1, epoch_num=100)

[ 549.70361328  368.43661499  367.50036621  366.86523438  366.42251587
  366.10870361  365.88290405  365.71755981  365.59423828  365.50033569]


In [57]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=1, epoch_num=100)

[  5.49696899e+02   6.33108378e+13   3.56993981e+29              nan
              nan              nan              nan              nan
              nan              nan]


In [58]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=0.1, epoch_num=1000)

[  549.47949219  1722.27734375  1712.2421875   1706.44970703  1561.6776123
   946.43664551   925.80541992   918.46594238   939.63635254   958.83605957]


In [59]:
result = mult_layers_nn(train_data=train_data, train_results=train_results, test_data=test_data, hidden_nodes_num=100,
                     learning_rate=0.01, epoch_num=1000)

[ 549.67004395  336.18185425  354.39868164  360.91418457  349.86746216
  351.91119385  353.91931152  343.92974854  325.59405518  335.49816895]


In [ ]:
compare_results(function=mult_layers_nn, test_results=train_results, parameter_name="epoch_num",
                list_of_values=[10, 25, 50, 100], train_data=train_data, train_results=train_results,
                test_data=train_data, learning_rate=0.0001, hidden_nodes_num=100)

In [ ]:
compare_results(function=mult_layers_nn, test_results=test_results, parameter_name="hidden_nodes_num",
                list_of_values=[50, 100], train_data=train_data, train_results=train_results,
                test_data=test_data, learning_rate=0.0001)